In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, GRU, SimpleRNN, Dense, Input, Bidirectional, LSTMCell, GRUCell, SimpleRNNCell
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

In [ ]:
# Define hyperparameters and dataset-related values
vocab_size = 10000
embedding_dim = 128
lstm_units = 64
gru_units = 64
rnn_units = 64
num_epochs = 10
batch_size = 64

In [ ]:
# Generate sample data (placeholders)
x_train = tf.random.uniform((1000, 50), minval=0, maxval=vocab_size, dtype=tf.int32)
y_train = tf.random.uniform((1000, 1), minval=0, maxval=2, dtype=tf.int32)
encoder_input_train = tf.random.uniform((1000, 20), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_input_train = tf.random.uniform((1000, 25), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_output_train = tf.random.uniform((1000, 25, vocab_size), minval=0, maxval=2, dtype=tf.float32)

In [ ]:
# Create a NestedCell Custom Layer
class NestedCell(Layer):
    def __init__(self, units, **kwargs):
        super(NestedCell, self).__init__(**kwargs)
        self.units = units
        self.lstm_cell = LSTMCell(self.units)
        self.gru_cell = GRUCell(self.units)
        self.rnn_cell = SimpleRNNCell(self.units)

    def call(self, inputs, states):
        return self.lstm_cell(inputs, states), self.gru_cell(inputs, states), self.rnn_cell(inputs, states)

In [ ]:
# Create instances of the different models
def create_embedding_lstm_model():
    input_layer = Input(shape=(None,))
    embedded = Embedding(vocab_size, embedding_dim)(input_layer)
    lstm_output = LSTM(lstm_units)(embedded)
    output_layer = Dense(1, activation='sigmoid')(lstm_output)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
# Create an Embedding + GRU + SimpleRNN + Dense model
def create_embedding_gru_rnn_model():
    input_layer = Input(shape=(None,))
    embedded = Embedding(vocab_size, embedding_dim)(input_layer)
    gru_output = GRU(gru_units, return_sequences=True)(embedded)
    rnn_output = SimpleRNN(rnn_units)(gru_output)
    output_layer = Dense(1, activation='sigmoid')(rnn_output)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
# Create an Encoder-Decoder model
def create_encoder_decoder_model():
    encoder_inputs = Input(shape=(None,))
    encoder_embedded = Embedding(vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(lstm_units, return_state=True)
    _, state_h, state_c = encoder_lstm(encoder_embedded)

    decoder_inputs = Input(shape=(None,))
    decoder_embedded = Embedding(vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedded, initial_state=[state_h, state_c])
    output_layer = Dense(vocab_size, activation='softmax')(decoder_outputs)

    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=output_layer)
    return model

In [ ]:
# Create a Bidirectional LSTM model
def create_bidirectional_lstm_model():
    input_layer = Input(shape=(None,))
    embedded = Embedding(vocab_size, embedding_dim)(input_layer)
    bidirectional_lstm = Bidirectional(LSTM(lstm_units))(embedded)
    output_layer = Dense(1, activation='sigmoid')(bidirectional_lstm)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
# Create a NestedCell Custom Layer
class NestedCell(Layer):
    def __init__(self, units, **kwargs):
        super(NestedCell, self).__init__(**kwargs)
        self.units = units
        self.lstm_cell = LSTMCell(self.units)
        self.gru_cell = GRUCell(self.units)
        self.rnn_cell = SimpleRNNCell(self.units)

    def call(self, inputs, states):
        return self.lstm_cell(inputs, states), self.gru_cell(inputs, states), self.rnn_cell(inputs, states)

In [ ]:
# Create instances of the different models
embedding_lstm_model = create_embedding_lstm_model()
embedding_gru_rnn_model = create_embedding_gru_rnn_model()
encoder_decoder_model = create_encoder_decoder_model()
bidirectional_lstm_model = create_bidirectional_lstm_model()

In [ ]:
# Compile the models
embedding_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
embedding_gru_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
encoder_decoder_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
bidirectional_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the models
embedding_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
embedding_gru_rnn_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
encoder_decoder_model.fit([encoder_input_train, decoder_input_train], decoder_output_train, epochs=num_epochs, batch_size=batch_size)
bidirectional_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
16/16 [==============================] - 47s 86ms/step - loss: 0.6933 - accuracy: 0.5070
Epoch 2/10
16/16 [==============================] - 1s 86ms/step - loss: 0.6491 - accuracy: 0.9490
Epoch 3/10
16/16 [==============================] - 1s 87ms/step - loss: 0.3430 - accuracy: 0.9570
Epoch 4/10
16/16 [==============================] - 2s 124ms/step - loss: 0.0766 - accuracy: 0.9890
Epoch 5/10
16/16 [==============================] - 2s 153ms/step - loss: 0.0249 - accuracy: 1.0000
Epoch 6/10
16/16 [==============================] - 2s 138ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 7/10
16/16 [==============================] - 2s 115ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 8/10
16/16 [==============================] - 1s 87ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 9/10
16/16 [==============================] - 1s 87ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 10/10
16/16 [==============================] - 1s 87ms/step - loss: 0.0022 - accuracy: 1.0000
Epoc

In [ ]:
# Print model summaries
print("Embedding + LSTM Model:")
embedding_lstm_model.summary()

Embedding + LSTM Model:
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_29 (Embedding)    (None, None, 128)         1280000   
                                                                 
 lstm_38 (LSTM)              (None, 64)                49408     
                                                                 
 dense_36 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print("\nEmbedding + GRU + SimpleRNN Model:")
embedding_gru_rnn_model.summary()


Embedding + GRU + SimpleRNN Model:
Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_41 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_30 (Embedding)    (None, None, 128)         1280000   
                                                                 
 gru_8 (GRU)                 (None, None, 64)          37248     
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 64)                8256      
                                                                 
 dense_37 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,325,569
Trainable params: 1,325,569
Non-trainable params: 0
_________________________________________________________________


In [ ]:
print("\nEncoder-Decoder Model:")
encoder_decoder_model.summary()


Encoder-Decoder Model:
Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_42 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_43 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_31 (Embedding)       (None, None, 128)    1280000     ['input_42[0][0]']               
                                                                                                  
 embedding_32 (Embedding)       (None, None, 128)    1280000     ['input_43[0][0]']               
                                                                   

In [ ]:
print("\nBidirectional LSTM Model:")
bidirectional_lstm_model.summary()


Bidirectional LSTM Model:
Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_44 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_33 (Embedding)    (None, None, 128)         1280000   
                                                                 
 bidirectional_11 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dense_39 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Generate sample data (placeholders)
x_train = tf.random.uniform((1000, 50), minval=0, maxval=vocab_size, dtype=tf.int32)
y_train = tf.random.uniform((1000, 1), minval=0, maxval=2, dtype=tf.int32)
encoder_input_train = tf.random.uniform((1000, 20), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_input_train = tf.random.uniform((1000, 25), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_output_train = tf.random.uniform((1000, 25, vocab_size), minval=0, maxval=2, dtype=tf.float32)

In [ ]:
# Train the models
embedding_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
embedding_gru_rnn_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
encoder_decoder_model.fit([encoder_input_train, decoder_input_train], decoder_output_train, epochs=num_epochs, batch_size=batch_size)
bidirectional_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
16/16 [==============================] - 1s 89ms/step - loss: 1.0334 - accuracy: 0.5100
Epoch 2/10
16/16 [==============================] - 2s 149ms/step - loss: 0.5305 - accuracy: 0.8260
Epoch 3/10
16/16 [==============================] - 2s 135ms/step - loss: 0.3598 - accuracy: 0.9670
Epoch 4/10
16/16 [==============================] - 1s 87ms/step - loss: 0.1666 - accuracy: 0.9920
Epoch 5/10
16/16 [==============================] - 1s 89ms/step - loss: 0.0577 - accuracy: 1.0000
Epoch 6/10
16/16 [==============================] - 1s 89ms/step - loss: 0.0252 - accuracy: 1.0000
Epoch 7/10
16/16 [==============================] - 1s 89ms/step - loss: 0.0154 - accuracy: 1.0000
Epoch 8/10
16/16 [==============================] - 1s 87ms/step - loss: 0.0111 - accuracy: 1.0000
Epoch 9/10
16/16 [==============================] - 1s 89ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 10/10
16/16 [==============================] - 1s 93ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 1